In [4]:
time_dirc = '1129_2141'

sem_pt_path = '/home/ghosnp/project/fix_space/origin/carla_dataset_tools/raw_data/record_2023_'+time_dirc+'/vehicle.tesla.model3.master/velodyne_semantic/'
image_path = '/home/ghosnp/project/fix_space/origin/carla_dataset_tools/raw_data/record_2023_'+time_dirc+'/vehicle.tesla.model3.master/image_2/'
save_path = '/home/ghosnp/project/fix_space/origin/carla_dataset_tools/raw_data/record_2023_'+time_dirc+'/vehicle.tesla.model3.master/vis_s2_score/'
# read the txt in sem_pt_path, draw the last float in last line into the same frame in image_path

import os
import cv2
# create save_path if not exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

# get the file list in sem_pt_path, end with .txt
file_list = os.listdir(sem_pt_path)
file_list = [f for f in file_list if f.endswith('.txt')]
file_list.sort()

# get the file list in image_path, same frame with sem_pt_path
image_list = os.listdir(image_path)
image_list = [f for f in image_list if f.endswith('.png')]
image_list.sort()
# filiter the image_list, drop the file that not in sem_pt_path
image_list = [f for f in image_list if f[:-4]+'.txt' in file_list]

# read the file in sem_pt_path, get the last float in last line
for file in file_list:
    with open(sem_pt_path+file, 'r') as f:
        lines = f.readlines()
        last_line = lines[-1]
        scores = last_line.split(' ')
        print(scores)
    # draw the score in the same frame in image_path
    textp = 'scan_enp: '+scores[1]+'; bev_enp: '+scores[2]+'; pf_scalar: '+scores[3]+'; s2_vector: '+scores[4]
    image_name = file[:-4]+'.png'
    image = cv2.imread(image_path+image_name)
    image = cv2.putText(image, textp, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.imwrite(save_path+image_name, image)
    # print(image_name)


['41', '0.6473024520629953', '0.00955778301241584', '0.02655952937690074', '0.6418332863409401\n']
['41', '0.9551263167266356', '0.028458268386508324', '0.026737233430687898', '0.6486105604912173\n']
['41', '0.9489856564039006', '0.028607740700299394', '0.027165101155965238', '0.6884048713979253\n']
['41', '0.9708139760813591', '0.030061206084298617', '0.035299417059484715', '0.6955469270809179\n']
['41', '0.9797219516472244', '0.029448524841842412', '0.05265356552442281', '0.6281710479866357\n']
['41', '1.0179178291828084', '0.03072523796835939', '0.08048289105108627', '0.5587790807841573\n']
['41', '1.0724981160474674', '0.03256775855564356', '0.14631893170341306', '0.6470483433160873\n']
['41', '1.134310738694589', '0.033987761621856044', '0.19711913907399267', '0.7252654299073118\n']
['41', '1.1522754722250843', '0.033450435917677206', '0.8425611928033805', '0.6785376137683313\n']
['71', '1.1549723740437263', '0.03303169852736916', '0.6571285451544847', '0.3377466484920749\n']
['61

In [7]:
# save the data(scan_enp, bev_enp, pf_scalar, pf_vector) in a csv file
import pandas as pd
import os
import cv2
import numpy as np


# time_dirc_list = ['1128_1836','1128_2219','1128_2256']
# labels_list = ['100-50','150-50','75-50']
# time_dirc_list = ['1129_2112','1129_2126','1129_2141','1129_2200']
# labels_list = ['75-50','100-50','125-50','150-50']
time_dirc_list = ['1129_2235']
labels_list = ['75-50','100-50','125-50','150-50']

label_valid = {'Car', 'Truck', 'Bus'}

idx, scan_entropy_list, bev_entropy_list, pf_scalar_list, pf_vector_list, s1_gt =[], [], [], [], [], []

for i in range(len(time_dirc_list)):
    time_dirc = time_dirc_list[i]
    sem_pt_path = '/home/ghosnp/project/fix_space/origin/carla_dataset_tools/raw_data/record_2023_'+time_dirc+'/vehicle.tesla.model3.master/velodyne_semantic/'
    file_list = os.listdir(sem_pt_path)
    file_list = [f for f in file_list if f.endswith('.txt')]
    file_list.sort()


    # the txt format is :
    # h, w, l, x, y, z, rot, lab, mes
    # and the last line is: scan_enp, bev_enp, pf_scalar, pf_vector
    # now, we need to calculate the count of lines whose dist(x**2+y**2) < 60, and save it into s1_gt
    for file in file_list:
        with open(sem_pt_path+file, 'r') as f:
            lines = f.readlines()
            last_line = lines[-1]
            scores = last_line.split(' ')

            scan_entropy_list.append(scores[1])
            bev_entropy_list.append(scores[2])
            pf_scalar_list.append(scores[3])
            pf_vector_list.append(scores[4].replace('\n',''))
            # ll.append(last_line)

            # calculate the s1_gt
            # count = 0
            # for line in lines[:-1]:
            #     line = line.split(' ')
            #     if line[7] in label_valid:
            #         x, y = float(line[3]), float(line[4])
            #         if x**2+y**2 < 60**2:
            #             count += 1
            s1_gt.append(scores[0])
            #idx.append(scores[0])
        


df = pd.DataFrame(columns=['scan_enp', 'bev_enp', 'pf_scalar', 'pf_vector','s1_gt'])

df['scan_enp'] = scan_entropy_list
df['bev_enp'] = bev_entropy_list
df['pf_scalar'] = pf_scalar_list
df['pf_vector'] = pf_vector_list
df['s1_gt'] = s1_gt

save_path = './s1s2_score.csv'

df.to_csv(save_path, index=False)
